In [6]:
# import pandas as pd
# import numpy as np

In [7]:
# gym_data = pd.read_csv("gym/gym_exercise_dataset.csv")
# gym_data.head()

In [8]:
# exe_data = pd.read_csv("gym/exercises.csv")
# exe_data.head()

In [9]:

# exe_data.columns

In [10]:
# df= exe_data['target'].unique()
# df

In [11]:
# exe_data.dtypes

In [12]:
# gym_data.columns

In [13]:
# import pandas as pd
# from sklearn.model_selection import train_test_split
# from sklearn.preprocessing import LabelEncoder
# from sklearn.neighbors import KNeighborsClassifier
# from sklearn.ensemble import RandomForestClassifier
# from sklearn.metrics import accuracy_score

# # Load the dataset
# # Assuming the dataset is a CSV
# df = pd.read_csv('gym/Exercises.csv')

# # Label encoding for categorical columns
# label_encoders = {}
# categorical_columns = ['bodyPart', 'equipment', 'target', 'secondaryMuscles/0', 'secondaryMuscles/1',
#                        'secondaryMuscles/2', 'secondaryMuscles/3', 'secondaryMuscles/4', 'secondaryMuscles/5']

# # Encode each categorical column
# for col in categorical_columns:
#     le = LabelEncoder()
#     df[col] = le.fit_transform(df[col].astype(str))  # Handle NaN values if necessary
#     label_encoders[col] = le  # Save the encoders for later use

# # Selecting features
# X = df[['bodyPart', 'equipment', 'target', 'secondaryMuscles/0', 'secondaryMuscles/1', 
#         'secondaryMuscles/2', 'secondaryMuscles/3', 'secondaryMuscles/4', 'secondaryMuscles/5']]

# # Target variable (target muscle)
# y = df['target']

# # Split the dataset into training and test sets
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)# Random Forest model
# rf = RandomForestClassifier(n_estimators=100, random_state=42)

# # Train the model
# rf.fit(X_train, y_train)

# # Predict on test set
# y_pred_rf = rf.predict(X_test)

# # Evaluate accuracy
# accuracy_rf = accuracy_score(y_test, y_pred_rf)
# print(f'Random Forest Accuracy: {accuracy_rf * 100:.2f}%')
# # Example: Recommend exercises similar to a specific target
# def recommend_exercises(target_input, k=5):
#     # Encode the target input using the label encoder
#     target_encoded = label_encoders['target'].transform([target_input])[0]
    
#     # Get the predicted probabilities for the input
#     probabilities = rf.predict_proba(X_test)
    
#     # # Get indices of top k exercises similar to the input
#     recommended_indices = np.argsort(probabilities[:, target_encoded])[-k:][::-1]
    
#     # # Decode the recommended target exercises
#     recommendations = df.iloc[recommended_indices]
    
#     # Return the recommended exercises
#     return recommendations

# # Example usage
# target_input = 'triceps'  # Provide a target muscle group
# recommended_exercises = recommend_exercises(target_input)
# print(recommended_exercises)





In [14]:
# # KNN model
# knn = KNeighborsClassifier(n_neighbors=5)

# # Train the model
# knn.fit(X_train, y_train)

# # Predict on test set
# y_pred_knn = knn.predict(X_test)

# # Evaluate accuracy
# accuracy_knn = accuracy_score(y_test, y_pred_knn)
# print(f'KNN Accuracy: {accuracy_knn * 100:.2f}%')



In [15]:
# import pandas as pd
# import numpy as np
# from sklearn.model_selection import train_test_split
# from sklearn.preprocessing import LabelEncoder
# from sklearn.ensemble import RandomForestClassifier
# from sklearn.metrics import accuracy_score
# import joblib  # For saving and loading the model
# import os

# # Load the dataset (update the path as needed)
# df = pd.read_csv('gym/exercises.csv')

# # Label encoding for categorical columns
# label_encoders = {}
# categorical_columns = ['bodyPart', 'equipment', 'target', 'secondaryMuscles/0', 'secondaryMuscles/1',
#                        'secondaryMuscles/2', 'secondaryMuscles/3', 'secondaryMuscles/4', 'secondaryMuscles/5']

# # Encode each categorical column
# for col in categorical_columns:
#     le = LabelEncoder()
#     df[col] = le.fit_transform(df[col].astype(str))  # Handle NaN values if necessary
#     label_encoders[col] = le  # Save the encoders for later use

# # Selecting features
# X = df[['bodyPart', 'equipment', 'target', 'secondaryMuscles/0', 'secondaryMuscles/1', 
#         'secondaryMuscles/2', 'secondaryMuscles/3', 'secondaryMuscles/4', 'secondaryMuscles/5']]

# # Target variable (target muscle)
# y = df['target']

# # Split the dataset into training and test sets
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# # Random Forest model
# rf = RandomForestClassifier(n_estimators=100, random_state=42)

# # Train the model
# rf.fit(X_train, y_train)

# # Predict on test set
# y_pred_rf = rf.predict(X_test)

# # Evaluate accuracy
# accuracy_rf = accuracy_score(y_test, y_pred_rf)
# print(f'Random Forest Accuracy: {accuracy_rf * 100:.2f}%')

# # Save the trained model as a .pkl file
# model_filename = 'exercise_recommender_model.pkl'
# joblib.dump(rf, model_filename)
# print(f'Model saved as {model_filename}')

# # Save the label encoders for future use
# encoder_filename = 'label_encoders.pkl'
# joblib.dump(label_encoders, encoder_filename)
# print(f'Label encoders saved as {encoder_filename}')

# # Function to recommend exercises similar to a specific target
# def recommend_exercises(target_input, k=5):
#     # Error handling for invalid inputs
#     if target_input not in label_encoders['target'].classes_:
#         return f"Invalid target: {target_input}. Available options: {label_encoders['target'].classes_}"
    
#     # Encode the target input using the label encoder
#     target_encoded = label_encoders['target'].transform([target_input])[0]
    
#     # Get the predicted probabilities for all test data
#     probabilities = rf.predict_proba(X_test)
    
#     # Get indices of top k exercises similar to the input
#     recommended_indices = np.argsort(probabilities[:, target_encoded])[-k:][::-1]
    
#     # Decode the recommended target exercises
#     recommendations = df.iloc[recommended_indices]
    
#     # Return the recommended exercises
#     return recommendations

# # Example usage
# target_input = 'quads'  # Provide a target muscle group
# recommended_exercises = recommend_exercises(target_input)
# print(recommended_exercises)


In [16]:
# import joblib
# # # Save the RandomForest model and label encoders
# joblib.dump(rf, 'exe_model.pkl')
# joblib.dump(label_encoders, 'exe_label_encoders.pkl')

In [17]:
# # Example: Recommend exercises similar to a specific target
# def recommend_exercises(target_input, k=5):
#     target_encoded = label_encoders['target'].transform([target_input])[0]
    
#     # Find the k-nearest neighbors
#     distances, indices = knn.kneighbors(X[X['target'] == target_encoded], n_neighbors=k)
    
#     # Get the recommended exercises
#     recommended_exercises = df.iloc[indices[0]]
#     return recommended_exercises[['name', 'target', 'bodyPart', 'equipment']]

# # Recommend exercises similar to 'biceps'
# recommend_exercises('biceps', k=5)


In [18]:
# # Feature importance from the random forest
# importances = rf.feature_importances_
# feature_names = X.columns

# # Display the feature importance
# for name, importance in zip(feature_names, importances):
#     print(f'{name}: {importance:.4f}')


In [19]:
# import pickle
# # Save the Random Forest model and label encoders
# with open('exercise_model.pkl', 'wb') as file:
#     pickle.dump({'model': rf, 'label_encoders': label_encoders}, file)

In [20]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

# Load the dataset
# Assuming the dataset is a CSV
df = pd.read_csv('./gym/Exersices.csv')

# Label encoding for categorical columns
label_encoders = {}
categorical_columns = ['bodyPart', 'equipment', 'target']

# Encode each categorical column
for col in categorical_columns:
    le = LabelEncoder()
    df[col] = le.fit_transform(df[col].astype(str))  # Handle NaN values if necessary
    label_encoders[col] = le  # Save the encoders for later use

# Selecting features
X = df[['bodyPart', 'equipment', 'target']]

# Target variable (target muscle)
y = df['target']

# Split the dataset into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)# Random Forest model
rf = RandomForestClassifier(n_estimators=100, random_state=42)

# Train the model
rf.fit(X_train, y_train)

# Predict on test set
y_pred_rf = rf.predict(X_test)

# Evaluate accuracy
accuracy_rf = accuracy_score(y_test, y_pred_rf)
print(f'Random Forest Accuracy: {accuracy_rf * 100:.2f}%')
# Example: Recommend exercises similar to a specific target
def recommend_exercises(target_input, k=5):
    # Encode the target input using the label encoder
    target_encoded = label_encoders['target'].transform([target_input])[0]
    
    # Get the predicted probabilities for the input
    probabilities = rf.predict_proba(X_test)
    
    # # Get indices of top k exercises similar to the input
    recommended_indices = np.argsort(probabilities[:, target_encoded])[-k:][::-1]
    
    # # Decode the recommended target exercises
    recommendations = df.iloc[recommended_indices]
    
    # Return the recommended exercises
    return recommendations

# Example usage
target_input = 'traps'  # Provide a target muscle group
recommended_exercises = recommend_exercises(target_input)
print(recommended_exercises)





Random Forest Accuracy: 99.25%
     Unnamed: 0  bodyPart  equipment                            name  target  \
152         152         0         16                   london bridge      18   
169         169         0         15             roller back stretch      15   
156         156         0          3              mixed grip chin-up      10   
153         153         0          3                 lower back curl      15   
62           62         0          5  cable rope elevated seated row      18   

                                          instructions  
152  ['Attach the rope to a high anchor point.', 'S...  
169  ['Start by sitting on the ground with your leg...  
156  ['Grab the pull-up bar with an underhand grip ...  
153  ['Lie flat on your stomach with your legs exte...  
62   ['Sit on the elevated seat facing the cable ma...  


In [21]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score

# Load the dataset
df = pd.read_csv('./gym/Exersices.csv')

# Label encoding for categorical columns
label_encoders = {}
categorical_columns = ['bodyPart', 'equipment', 'target']

# Encode each categorical column
for col in categorical_columns:
    le = LabelEncoder()
    df[col] = le.fit_transform(df[col].astype(str))  # Handle NaN values if necessary
    label_encoders[col] = le  # Save the encoders for later use

# Selecting features
X = df[['bodyPart', 'equipment', 'target']]
y = df['target']  # Target variable

# Standardize features for KNN
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Split the dataset into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# Define the KNN model
k = 5
knn = KNeighborsClassifier(n_neighbors=k)

# Train the model
knn.fit(X_train, y_train)

# Predict on test set to evaluate accuracy
y_pred_knn = knn.predict(X_test)
accuracy_knn = accuracy_score(y_test, y_pred_knn)
print(f'KNN Accuracy: {accuracy_knn * 100:.2f}%')

# Recommendation function using KNN
def recommend_exercises(target_input, k=5):
    # Encode the target input using the label encoder
    target_encoded = label_encoders['target'].transform([target_input])[0]
    
    # Create a sample input with the encoded target value
    sample_input = X[X['target'] == target_encoded].iloc[0]  # Get one example with this target
    
    # Standardize the sample input for KNN compatibility
    sample_input_scaled = scaler.transform([sample_input])
    
    # Find the k nearest neighbors
    distances, indices = knn.kneighbors(sample_input_scaled, n_neighbors=k)
    
    # Decode the recommended target exercises
    recommendations = df.iloc[indices[0]]
    
    # Return the recommended exercises
    return recommendations

# Example usage
target_input = 'traps'  # Provide a target muscle group
recommended_exercises = recommend_exercises(target_input)
print(recommended_exercises)


KNN Accuracy: 94.34%
     Unnamed: 0  bodyPart  equipment  \
798         798         7          6   
166         166         0         11   
781         781         7          6   
456         456         4          6   
491         491         5          3   

                                                  name  target  \
798                dumbbell one arm seated hammer curl       3   
166                  reverse grip machine lat pulldown      10   
781  dumbbell lying one arm supinated triceps exten...      17   
456                     dumbbell single leg calf raise       4   
491                                  neck side stretch      11   

                                          instructions  
798  ['Sit on a bench with your back straight and f...  
166  ['Adjust the seat height and position yourself...  
781  ['Lie flat on a bench with your back and head ...  
456  ['Stand on the edge of a step or platform with...  
491  ['Stand or sit up straight with your shoulders...  

c:\Users\Dell\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:465: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
